# Collaborative filtering

In [141]:
#librairies
# classic Librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import shutil

#loading embeddings
import pickle

#sklearn utils
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import NearestNeighbors

#surprise
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split #really different from sklearn's ? 
from surprise import accuracy

In [142]:
#check if .ipynb files have been generated : len should be 385
path = "data/archive/clicks/clicks"
list_of_files = os.listdir(path)
print(len(list_of_files))

385


In [143]:
#concat all files in one
dfs = []
for file in list_of_files:
    file_path = os.path.join(path, file)
    dfs.append(pd.read_csv(file_path))
clicks_full = pd.concat(dfs, ignore_index=True)

In [144]:
print("Full size data set :", clicks_full.shape)

Full size data set : (2988181, 12)


In [145]:
#loading sample
data_sample = pd.read_csv("data/archive/clicks_sample.csv")
data_sample.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


In [146]:
#create rating
ratings = (
    data_sample
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)


In [147]:
ratings.head()

,user_id,click_article_id,rating
0,0,68866,1
1,0,157541,1
2,1,96663,1
3,1,235840,1
4,2,30970,1


In [148]:
reader = Reader(rating_scale=(1, ratings.rating.max()))
data   = Dataset.load_from_df(ratings[["user_id","click_article_id","rating"]], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [149]:
print(ratings["rating"].value_counts().sort_index())


rating
1    1883
Name: count, dtype: int64


In [150]:
algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)
preds = algo.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))

RMSE: 0.0000
RMSE: 0.0
MAE:  0.0000
MAE:  0.0


In [151]:
#EDA
# 2.2 Aperçu
print(data_sample.shape)
display(data_sample.head())
print(data_sample.info())

# 2.3 Valeurs manquantes / doublons
print("Nulls :", data_sample.isna().sum().sum())
print("Doublons :", data_sample.duplicated().sum())

# 2.4 Nombre d’utilisateurs / articles
print("Users uniques :", data_sample["user_id"].nunique())
print("Articles uniques :", data_sample["click_article_id"].nunique())


(1883, 12)


,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   user_id              1883 non-null   int64
 1   session_id           1883 non-null   int64
 2   session_start        1883 non-null   int64
 3   session_size         1883 non-null   int64
 4   click_article_id     1883 non-null   int64
 5   click_timestamp      1883 non-null   int64
 6   click_environment    1883 non-null   int64
 7   click_deviceGroup    1883 non-null   int64
 8   click_os             1883 non-null   int64
 9   click_country        1883 non-null   int64
 10  click_region         1883 non-null   int64
 11  click_referrer_type  1883 non-null   int64
dtypes: int64(12)
memory usage: 176.7 KB
None
Nulls : 0
Doublons : 0
Users uniques : 707
Articles uniques : 323


In [152]:
# 3.2 Agréger en comptant les clics par (user, article)
ratings = (
    data_sample
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)

print(ratings.shape)
display(ratings.head())
print(ratings["rating"].describe())


(1883, 3)


,user_id,click_article_id,rating
0,0,68866,1
1,0,157541,1
2,1,96663,1
3,1,235840,1
4,2,30970,1


count    1883.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: rating, dtype: float64


In [157]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# 4.1 Création de l’objet Dataset
reader = Reader(rating_scale=(ratings["rating"].min(), ratings["rating"].max()))
data = Dataset.load_from_df(ratings[["user_id","click_article_id","rating"]], reader)

# 4.2 Split train/test
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 4.3 Entraînement d’un SVD de base
algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)

# 4.4 Évaluation
preds = algo.test(testset)
print("RMSE :", accuracy.rmse(preds))
print("MAE  :", accuracy.mae(preds))


RMSE: 0.0000
RMSE : 0.0
MAE:  0.0000
MAE  : 0.0


In [158]:
from collections import defaultdict
import numpy as np

def precision_recall_at_k(predictions, k=5, threshold=1.0):
    # Regrouper les prédictions par user
    user_pred = defaultdict(list)
    user_true = defaultdict(set)
    for uid, iid, true_r, est, _ in predictions:
        user_pred[uid].append((iid, est))
        if true_r >= threshold:
            user_true[uid].add(iid)

    precisions, recalls = [], []
    for uid, preds in user_pred.items():
        # top-k par score estimé
        top_k = [iid for (iid, _) in sorted(preds, key=lambda x: x[1], reverse=True)[:k]]
        n_rel = len(user_true[uid])
        n_rec_k = len([i for i in top_k if i in user_true[uid]])
        precisions.append(n_rec_k / k)
        recalls.append(n_rec_k / n_rel if n_rel else 0)

    return np.mean(precisions), np.mean(recalls)

prec, rec = precision_recall_at_k(preds, k=5, threshold=1.0)
print(f"Precision@5 = {prec:.4f}, Recall@5 = {rec:.4f}")


Precision@5 = 0.2591, Recall@5 = 1.0000


In [159]:
data_meta = pd.read_csv("data/archive/articles_metadata.csv")
# 1. Vérification de la correspondance des IDs
click_ids = set(clicks_full["click_article_id"].unique())
meta_ids  = set(data_meta["article_id"].unique())

n_click_ids   = len(click_ids)
n_meta_ids    = len(meta_ids)
n_common_ids  = len(click_ids & meta_ids)

print(f"IDs uniques dans clicks : {n_click_ids}")
print(f"IDs uniques dans meta   : {n_meta_ids}")
print(f"IDs en commun           : {n_common_ids} ({n_common_ids/n_click_ids:.1%} des IDs de clics)")

# 2. IDs présents dans clicks mais pas dans meta
missing_in_meta = click_ids - meta_ids
print(f"IDs en clicks mais pas en meta : {len(missing_in_meta)}")
print("Quelques exemples :", list(missing_in_meta)[:10])

# 3. IDs présents dans meta mais pas dans clicks
missing_in_clicks = meta_ids - click_ids
print(f"IDs en meta mais pas en clicks : {len(missing_in_clicks)}")
print("Quelques exemples :", list(missing_in_clicks)[:10])

IDs uniques dans clicks : 46033
IDs uniques dans meta   : 364047
IDs en commun           : 46033 (100.0% des IDs de clics)
IDs en clicks mais pas en meta : 0
Quelques exemples : []
IDs en meta mais pas en clicks : 318014
Quelques exemples : [0, 1, 2, 4, 5, 6, 7, 8, 9, 10]


Which implies that users have read the same articles multiples times, and that some articles have never been read

Sounds strange and inaccurate

Retrain now with full dataset

In [161]:
# 3.2 Agréger en comptant les clics par (user, article)
ratings = (
    clicks_full
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)

print(ratings.shape)
display(ratings.head())
print(ratings["rating"].describe())


(2950710, 3)


,user_id,click_article_id,rating
0,0,68866,1
1,0,87205,1
2,0,87224,1
3,0,96755,1
4,0,157541,1


count    2.950710e+06
mean     1.012699e+00
std      1.344699e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      3.300000e+01
Name: rating, dtype: float64


In [162]:
ratings["rating"].unique()

array([ 1,  2,  5,  4,  3,  6,  7,  8, 10, 13,  9, 17, 12, 30, 31, 33, 16],
      dtype=int64)

In [163]:
#4(Optionnel) Calculer un dwell_time pour un rating pondéré
clicks_full = clicks_full.sort_values(
    ["user_id","session_id","click_timestamp"]
)
clicks_full["next_ts"] = (
    clicks_full
    .groupby(["user_id","session_id"])["click_timestamp"]
    .shift(-1)
)
clicks_full["dwell_time"] = (
    clicks_full["next_ts"] - clicks_full["click_timestamp"]
).fillna(0)

# 5. Construire le DataFrame de ratings sur counts
ratings_count = (
    clicks_full
    .groupby(["user_id","click_article_id"])
    .size()
    .reset_index(name="rating")
)
print(ratings_count["rating"].describe())

# 6. Ou bien sur dwell_time
ratings_time = (
    clicks_full
    .groupby(["user_id","click_article_id"])["dwell_time"]
    .sum()
    .reset_index(name="rating")
)
print(ratings_time["rating"].describe())

C:\Users\kanam\AppData\Local\Temp\ipykernel_24900\1109433486.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)


count    2.950710e+06
mean     1.012699e+00
std      1.344699e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      3.300000e+01
Name: rating, dtype: float64
count    2.950710e+06
mean     2.974197e+05
std      4.313964e+06
min      0.000000e+00
25%      0.000000e+00
50%      3.000000e+04
75%      4.452700e+04
max      1.212149e+09
Name: rating, dtype: float64


In [164]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Choix du rating
ratings = ratings_count  # ou ratings_time

reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data   = Dataset.load_from_df(
    ratings[["user_id","click_article_id","rating"]],
    reader
)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)

preds = algo.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))


RMSE: 0.1342
RMSE: 0.13417800012623615
MAE:  0.0260
MAE:  0.025957099979260493


In [172]:
#dwell time

# 2. Calculer le dwell_time (temps passé sur chaque article au sein de chaque session)
clicks_full = clicks_full.sort_values(
    ["user_id", "session_id", "click_timestamp"]
)

# Pour chaque clic, on prend le timestamp du clic suivant dans la même session
clicks_full["next_ts"] = (
    clicks_full
    .groupby(["user_id", "session_id"])["click_timestamp"]
    .shift(-1)
)

# dwell_time = différence au clic suivant, ou 0 si c'est le dernier clic de la session
clicks_full["dwell_time"] = (
    clicks_full["next_ts"] - clicks_full["click_timestamp"]
).fillna(0)

# 3. Construire le DataFrame de ratings sur la somme des dwell_time par (user, article)
ratings_time = (
    clicks_full
    .groupby(["user_id", "click_article_id"])["dwell_time"]
    .sum()
    .reset_index(name="rating")
)

# (Optionnel) Appliquer une transformation log+1 pour atténuer les valeurs extrêmes
ratings_time["rating"] = np.log1p(ratings_time["rating"])

# 4. Charger ces ratings dans Surprise et entraîner un modèle SVD
reader = Reader(rating_scale=(ratings_time["rating"].min(), ratings_time["rating"].max()))
data   = Dataset.load_from_df(
    ratings_time[["user_id", "click_article_id", "rating"]],
    reader
)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

algo = SVD(n_factors=50, random_state=42)
algo.fit(trainset)

# 5. Évaluer sur le testset (RMSE / MAE)
preds = algo.test(testset)
print("RMSE:", accuracy.rmse(preds))
print("MAE: ", accuracy.mae(preds))

# 6. Exemple de prédiction pour un utilisateur & un article donné
#    Format : algo.predict(uid, iid, r_ui=None, verbose=True)
#    - uid = ID utilisateur (doit exister dans trainset ou non ; Surprise peut faire du cold-start)
#    - iid = ID article (doit exister dans trainset ou non)
#    - r_ui = rating réel si connu, sinon None
#    - verbose = True pour afficher la prédiction détaillée

user_id    = 1234    # remplacer par un user_id de ton choix
article_id = 157541  # remplacer par un click_article_id de ton choix

prediction = algo.predict(uid=user_id, iid=article_id, r_ui=None, verbose=True)
# prediction.est est la valeur prédite (log(dwell_time+1)) pour ce couple (user_id, article_id)

# Si tu veux récupérer la valeur sous forme de dwell_time estimé, fais :
log_est = prediction.est
dwell_time_est = np.expm1(log_est)  # inverse de np.log1p
print(f"Estimation de dwell_time pour user {user_id}, article {article_id}: {dwell_time_est:.2f} ms")



C:\Users\kanam\AppData\Local\Temp\ipykernel_24900\3843021599.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)


RMSE: 6.0066
RMSE: 6.006615024680338
MAE:  5.1406
MAE:  5.140568493369283
user: 1234       item: 157541     r_ui = None   est = 7.13   {'was_impossible': False}
Estimation de dwell_time pour user 1234, article 157541: 1246.00 ms
